In [69]:
import os
import io
import sys
import time
import requests
import platform
import subprocess
import numpy as np
from IPython.display import Markdown, display, update_display
import gradio as gr

In [70]:
EXAMPLE_PI = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(10_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [71]:
EXAMPLE_MATRIX = """
import time
import numpy as np

def matrix_multiply(size):
    # Create two random matrices
    matrix_a = np.random.rand(size, size)
    matrix_b = np.random.rand(size, size)
    
    start_time = time.time()
    result = np.matmul(matrix_a, matrix_b)
    end_time = time.time()
    
    # Calculate the sum to verify the computation
    sum_value = np.sum(result)
    
    print(f"Matrix multiplication of {size}x{size} matrices")
    print(f"Result sum: {sum_value:.6f}")
    print(f"Execution Time: {(end_time - start_time):.6f} seconds")

matrix_multiply(500)
"""


In [72]:
EXAMPLES = {
    "Pi Calculation": EXAMPLE_PI,
    "Matrix Multiplication": EXAMPLE_MATRIX
}

# Helper functions
def write_output(code, file_name):
    """Write the generated code to a file."""
    with open(file_name, "w") as f:
        f.write(code)

def system_message_for(language):
    """Create a system message tailored for the requested language."""
    return (
        f"You are an assistant that reimplements Python code in high-performance {language.upper()} for an M1 Mac. "
        f"Respond only with {language.upper()} code; do not explain your work other than occasional comments. "
        "Pay attention to number types to ensure no overflows and include all necessary packages.\n\n"
    )

def user_prompt_for(python):
    """Generate the user prompt."""
    return (
        "Rewrite this Python code in the requested language with the fastest possible implementation that produces "
        "identical output in the least time. Use appropriate syntax for the language.\n\n" + python
    )


In [73]:
def execute_python(code):
    """
    Execute Python code dynamically and capture its output.
    """
    output = io.StringIO()
    try:
        old_stdout = sys.stdout
        sys.stdout = output  # Redirect standard output to the StringIO object
        start_time = time.time()
        exec(code, {"np": np, "time": time})  # Execute code with minimal context
        end_time = time.time()
        execution_time = end_time - start_time
    except Exception as e:
        return f"Error during execution: {str(e)}"
    finally:
        sys.stdout = old_stdout  # Restore standard output

    result = output.getvalue()
    if "Execution Time:" not in result:
        result += f"\nExecution Time: {execution_time:.6f} seconds"
    return result


In [74]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [75]:
def execute_cpp(code):
    """Compile and execute C++ code."""
    write_output(code, "optimized.cpp")
    is_windows = platform.system() == "Windows"
    
    try:
        # Compile command with optimizations
        if is_windows:
            compile_cmd = ["g++", "-Ofast", "-std=c++17", "-o", "optimized.exe", "optimized.cpp"]
            run_cmd = ["optimized.exe"]
        else:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=native", "-o", "optimized", "optimized.cpp"]
            run_cmd = ["./optimized"]
            
        # Compile
        compile_result = subprocess.run(
            compile_cmd, check=True, text=True, capture_output=True, shell=is_windows
        )
        
        # Run
        start_time = time.time()
        run_result = subprocess.run(
            run_cmd, check=True, text=True, capture_output=True, shell=is_windows
        )
        end_time = time.time()
        
        # Add execution time if not included in output
        output = run_result.stdout
        if "Execution Time:" not in output:
            output += f"\nExecution Time: {(end_time - start_time):.6f} seconds"
        
        return output
    except subprocess.CalledProcessError as e:
        return f"Error during compilation or execution:\n{e.stderr}"


In [76]:
def execute_java(code):
    """Compile and execute Java code dynamically."""
    # Determine the class name from the code
    import re
    class_pattern = re.compile(r"public\s+class\s+(\w+)")
    match = class_pattern.search(code)
    
    if match:
        class_name = match.group(1)
    else:
        class_name = "Optimized"  # Default class name
        
    file_name = f"{class_name}.java"
    write_output(code, file_name)
    
    is_windows = platform.system() == "Windows"
    
    try:
        # Compile the Java code
        compile_cmd = ["javac", file_name]
        subprocess.run(compile_cmd, check=True, text=True, capture_output=True, shell=is_windows)
        
        # Run the compiled Java program
        run_cmd = ["java", class_name]
        start_time = time.time()
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True, shell=is_windows)
        end_time = time.time()
        
        # Add execution time if not included in output
        output = run_result.stdout
        if "Execution Time:" not in output:
            output += f"\nExecution Time: {(end_time - start_time):.6f} seconds"
            
        return output
    except subprocess.CalledProcessError as e:
        return f"Error during compilation or execution:\n{e.stderr}"


In [77]:
def ollama_stream(python, language="cpp"):
    """Generate code using the Ollama API with streaming."""
    if language not in ["cpp", "java"]:
        raise ValueError("Invalid language specified. Choose 'cpp' or 'java'.")
    
    # Set up the prompt for code conversion
    system_message = system_message_for(language)
    user_message = user_prompt_for(python)
    
    # Streaming response with Ollama API
    url = "http://localhost:11434/api/generate"
    payload = {# change model if needed here
        "model": "llama3.2:1b",  # Using the codellama model from Ollama
        "prompt": f"{system_message}\n\n{user_message}",
        "stream": False  # Set to True for streaming, False for complete response
    }
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        
        if payload["stream"]:
            # This would handle streaming if needed
            full_response = ""
            for line in response.iter_lines():
                if line:
                    chunk = response.json()
                    if "response" in chunk:
                        full_response += chunk["response"]
                        yield full_response
        else:
            # Return the full response
            result = response.json()
            yield result["response"]
            
    except requests.exceptions.RequestException as e:
        yield f"Error connecting to Ollama API: {str(e)}\n\nPlease make sure Ollama is running with the llama model."


In [78]:
def deepseek_stream(python, language="cpp"):
    """Generate code using a small DeepSeek model via Hugging Face."""
    from huggingface_hub import InferenceClient
    
    if language not in ["cpp", "java"]:
        raise ValueError("Invalid language specified. Choose 'cpp' or 'java'.")
    
    # Set up the prompt for code conversion
    system_message = system_message_for(language)
    user_message = user_prompt_for(python)
    
    try:
        client = InferenceClient("deepseek-ai/deepseek-coder-1.3b-base")
        inputs = f"{system_message}\n\n{user_message}"
        
        # Stream the output
        full_response = ""
        for response in client.text_generation(
            inputs, 
            max_new_tokens=4096,
            stream=True,
            details=False,
            typical_p=0.2,
            temperature=0.1
        ):
            full_response += response
            yield full_response
            
    except Exception as e:
        yield f"Error using DeepSeek model: {str(e)}\n\nPlease check your internet connection or Hugging Face API access."


In [79]:
def optimize(python, model="Ollama", language="cpp"):
    """
    Optimize the given Python code using the specified model and generate the output
    in the requested programming language.

    Args:
        python (str): The Python code to optimize.
        model (str): The model to use ("Ollama" or "DeepSeek").
        language (str): The target programming language ("cpp" or "java").

    Yields:
        str: The streamed output of the generated code.
    """
    if model == "Ollama":
        result = ollama_stream(python, language=language)
    elif model == "DeepSeek":
        result = deepseek_stream(python, language=language)
    else:
        raise ValueError("Unknown model. Please choose 'Ollama' or 'DeepSeek'.")

    for stream_so_far in result:
        yield stream_so_far


In [80]:
css = """
:root {
    --python-color: #306998;
    --cpp-color: #00599C;
    --java-color: #b07219;
}

.gradio-container {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}

.app-header {
    text-align: center;
    margin-bottom: 2rem;
    background: linear-gradient(90deg, var(--python-color), var(--cpp-color), var(--java-color));
    padding: 1rem;
    border-radius: 8px;
    color: white;
}

.code-box {
    border-radius: 8px;
    border: 2px solid #ddd;
    font-family: 'Consolas', 'Monaco', monospace;
}

.python-box { border-color: var(--python-color); }
.cpp-box { border-color: var(--cpp-color); }
.java-box { border-color: var(--java-color); }

.python-result { background-color: rgba(48, 105, 152, 0.1); }
.cpp-result { background-color: rgba(0, 89, 156, 0.1); }
.java-result { background-color: rgba(176, 114, 25, 0.1); }

.btn {
    border-radius: 6px;
    font-weight: bold;
    transition: all 0.3s ease;
}

.python-btn { background-color: var(--python-color); color: white; }
.cpp-btn { background-color: var(--cpp-color); color: white; }
.java-btn { background-color: var(--java-color); color: white; }

.btn:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 8px rgba(0,0,0,0.2);
}

.comparison-table {
    width: 100%;
    border-collapse: collapse;
    margin-top: 1rem;
}

.comparison-table th, .comparison-table td {
    border: 1px solid #ddd;
    padding: 8px;
    text-align: center;
}

.comparison-table th {
    background-color: #f2f2f2;
}
"""


In [81]:
def create_ui():
    with gr.Blocks(css=css) as ui:
        gr.HTML("""
        <div class="app-header">
            <h1>Code Converter and Performance Analyzer</h1>
            <p>Convert Python code to C++ or Java and compare execution times</p>
        </div>
        """)
        
        with gr.Row():
            example_dropdown = gr.Dropdown(
                choices=list(EXAMPLES.keys()),
                label="Select Example Code",
                value="Pi Calculation"
            )
            
            model_dropdown = gr.Dropdown(
                choices=["Ollama", "DeepSeek"],
                label="Select AI Model",
                value="Ollama",
                info="Ollama requires local installation, DeepSeek uses Hugging Face API"
            )
        
        with gr.Row():
            python_code = gr.Code(
                label="Python Code",
                language="python",
                value=EXAMPLES["Pi Calculation"],
                elem_classes=["code-box", "python-box"]
            )
            
        with gr.Row():
            with gr.Column():
                convert_cpp_btn = gr.Button("Convert to C++", elem_classes=["btn", "cpp-btn"])
                cpp_code = gr.Code(
                    label="C++ Code",
                    language="cpp",
                    elem_classes=["code-box", "cpp-box"]
                )
                cpp_run_btn = gr.Button("Run C++", elem_classes=["btn", "cpp-btn"])
                cpp_output = gr.TextArea(
                    label="C++ Output",
                    elem_classes=["cpp-result"]
                )
                
            with gr.Column():
                convert_java_btn = gr.Button("Convert to Java", elem_classes=["btn", "java-btn"])
                java_code = gr.Code(
                    label="Java Code",
                    language="python",
                    elem_classes=["code-box", "java-box"]
                )
                java_run_btn = gr.Button("Run Java", elem_classes=["btn", "java-btn"])
                java_output = gr.TextArea(
                    label="Java Output",
                    elem_classes=["java-result"]
                )
        
        with gr.Row():
            python_run_btn = gr.Button("Run Python", elem_classes=["btn", "python-btn"])
            python_output = gr.TextArea(
                label="Python Output",
                elem_classes=["python-result"]
            )
            
        with gr.Row():
            gr.HTML("""
            <div>
                <h3>Performance Comparison</h3>
                <p>Run all three languages to see execution time comparison</p>
            </div>
            """)
        
        with gr.Row():
            compare_btn = gr.Button("Compare Performance", elem_classes=["btn"])
            comparison_results = gr.HTML(label="Comparison Results")
            
        # Example dropdown event
        example_dropdown.change(
            fn=lambda example: EXAMPLES[example],
            inputs=[example_dropdown],
            outputs=[python_code]
        )
        
        # Convert buttons
        convert_cpp_btn.click(
            fn=lambda code, model: next(optimize(code, model, "cpp")),
            inputs=[python_code, model_dropdown],
            outputs=[cpp_code]
        )
        
        convert_java_btn.click(
            fn=lambda code, model: next(optimize(code, model, "java")),
            inputs=[python_code, model_dropdown],
            outputs=[java_code]
        )
        
        # Run buttons
        python_run_btn.click(
            fn=execute_python,
            inputs=[python_code],
            outputs=[python_output]
        )
        
        cpp_run_btn.click(
            fn=execute_cpp,
            inputs=[cpp_code],
            outputs=[cpp_output]
        )
        
        java_run_btn.click(
            fn=execute_java,
            inputs=[java_code],
            outputs=[java_output]
        )
        
        # Compare performance function
        def compare_performance(python_code, cpp_code, java_code, python_output, cpp_output, java_output):
            # Extract execution times
            def extract_time(output):
                import re
                time_pattern = re.compile(r"Execution Time: (\d+\.\d+) seconds")
                match = time_pattern.search(output)
                if match:
                    return float(match.group(1))
                return None
            
            # If outputs are empty, execute the code
            if not python_output:
                python_output = execute_python(python_code)
            if not cpp_output and cpp_code:
                cpp_output = execute_cpp(cpp_code)
            if not java_output and java_code:
                java_output = execute_java(java_code)
            
            # Extract times
            python_time = extract_time(python_output) if python_output else None
            cpp_time = extract_time(cpp_output) if cpp_output else None
            java_time = extract_time(java_output) if java_output else None
            
            # Create comparison table
            table = "<table class='comparison-table'>"
            table += "<tr><th>Language</th><th>Execution Time (seconds)</th><th>Speedup vs Python</th></tr>"
            
            # Python row
            table += f"<tr><td>Python</td><td>{python_time:.6f if python_time else 'N/A'}</td><td>1.00x</td></tr>"
            
            # C++ row
            if cpp_time:
                speedup = python_time / cpp_time if python_time else 0
                table += f"<tr><td>C++</td><td>{cpp_time:.6f}</td><td>{speedup:.2f}x</td></tr>"
            else:
                table += "<tr><td>C++</td><td>N/A</td><td>N/A</td></tr>"
            
            # Java row
            if java_time:
                speedup = python_time / java_time if python_time else 0
                table += f"<tr><td>Java</td><td>{java_time:.6f}</td><td>{speedup:.2f}x</td></tr>"
            else:
                table += "<tr><td>Java</td><td>N/A</td><td>N/A</td></tr>"
            
            table += "</table>"
            
            # Determine fastest language
            times = {}
            if python_time: times["Python"] = python_time
            if cpp_time: times["C++"] = cpp_time
            if java_time: times["Java"] = java_time
            
            if times:
                fastest = min(times, key=times.get)
                table += f"<p><strong>Fastest language:</strong> {fastest} ({times[fastest]:.6f} seconds)</p>"
            
            return table, python_output, cpp_output, java_output
        
        # Compare button
        compare_btn.click(
            fn=compare_performance,
            inputs=[python_code, cpp_code, java_code, python_output, cpp_output, java_output],
            outputs=[comparison_results, python_output, cpp_output, java_output]
        )
        
        return ui

In [82]:
import io
import sys

def execute_python(code):
    """
    Execute Python code dynamically and capture its output.

    Args:
        code (str): The Python code to execute.

    Returns:
        str: The captured standard output of the executed code.

    Raises:
        Exception: If the execution of the code raises an error.
    """
    output = io.StringIO()
    try:
        sys.stdout = output  # Redirect standard output to the StringIO object
        exec(code, {})  # Execute code with an empty global context for safety
    except Exception as e:
        return f"Error during execution: {str(e)}"
    finally:
        sys.stdout = sys.__stdout__  # Restore standard output

    return output.getvalue()


In [83]:
# Main function to start the app
def main():
    ui = create_ui()
    ui.launch(debug=True)

if __name__ == "__main__":
    main()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
